

## Twiter Object Documentation

#### Twitter Source Documentation for info on tweet objects:

https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/tweet

https://developer.twitter.com/en/docs/twitter-api/tweets/lookup/api-reference/get-tweets


#### Tweet object structure:
-- Tweet object has long list of root level fields such as id, text, created_at

-- Tweet object is parent to several child objects including user, media, poll, and place

-- Use field paramater "tweet.fields" when requesting root level fields on tweet object

- tweet.fields:attachments, author_id, context_annotations, conversation_id, created_at, 
entities, geo, id, in_reply_to_user_id, lang,non_public_metrics, public_metrics, organic_metrics, 
promoted_metrics, possibly_sensitive, referenced_tweets, source, text, withheld


-- Child object paramaters (query paramaters):

- media.fields:  duration_ms, height, media_key, preview_image_url, type, url, width, public_metrics, 
non_public_metrics, organic_metrics, promoted_metrics

- place.fields:  contained_within, country, country_code, full_name, geo, id, name, place_type

- poll.fields:  duration_minutes, end_datetime, id, options, voting_status

- user.fields:  created_at, description, entities, id, location, name, 
pinned_tweet_id, profile_image_url, protected, public_metrics, url, username, verified, withheld

--Other query paramaters:

- expansions:  attachments.poll_ids, attachments.media_keys, author_id, entities.mentions.username, 
geo.place_id, in_reply_to_user_id, referenced_tweets.id, referenced_tweets.id.author_id



In [2]:
## Import dependencies

# Utilities
import requests
import json
import pandas as pd
import time
import pprint

# Database 
import pymongo
from pymongo import MongoClient

# Keys
from config import bearer_token


In [3]:
##  Set authorization for Twitter     

headers = {"Authorization": "Bearer {}".format(bearer_token)}

In [4]:
## Set URL and query paramaters; create response object

# Set query keywords
query = "covid -is:retweet lang:en"

#Set place fields to return place location if available 
place_fields = "place.fields=contained_within,country,country_code,geo"

# Set tweet fields to return
tweet_fields = "tweet.fields=author_id,lang,public_metrics,created_at"

# Set max results per page
max_results = 'max_results=100'

# Set initial value of next token 
next = ''

# Set URL
url = "https://api.twitter.com/2/tweets/search/recent?query={}&{}&{}&{}".format(query, place_fields, tweet_fields,
                                                                                max_results, next)

# Create response object
response = requests.request("GET", url, headers=headers)


In [5]:
# Confirm length of json repsonse; expect length of 100 (100 tweets)
len(response.json()['data'])

100

In [6]:
# Review json response
response.json()['data'][0]

{'lang': 'en',
 'author_id': '2993718043',
 'created_at': '2020-11-22T05:20:09.000Z',
 'id': '1330380527640211461',
 'text': '@MeidasTouch @johncusack I call bullshit on any of them having covid. It’s a ploy to try and show the country covid is nothing. My husband currently still has the virus and is suffering. Fuck their bullshit lies.',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0}}

In [ ]:
# -------------------------------
# Test mongo insertion
# -------------------------------

In [18]:
# Establish Mongo connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Connect to database
db = client.TweetAPI_db

# Define Mongo Collection to store data
collection = db.tweets_db

In [19]:
# Insert one value into database

# db.tweets_db.insert_one(response.json()['data'][0])

In [20]:
# Check insertion

# db.tweets_db.find_one()

{'_id': ObjectId('5fb91a0869accb7704c96884'),
 'id': '1330143487581450246',
 'author_id': '1318309879417774080',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0},
 'text': 'Allah created people and the people create virus called covid-19.2020 is a year of disaster and I shaĺl never forget my entire live.Orphan trees has great story and all hyderabad people knew orphan trees. https://t.co/4I98EL6gTz',
 'created_at': '2020-11-21T13:38:14.000Z',
 'lang': 'en'}

In [21]:
# Insert entire response.json results (100 tweets) into collection

# db.tweets_db.insert_many(response.json()['data'])

In [53]:
# Clear collection

# db.tweets_db.drop()

In [ ]:
# -------------------------------
# Running mongo insertion
# -------------------------------

In [54]:
## Create while loop to paginate through JSON responses so long as next_token in return in metadata at end of file

# NOTE:  Replace counter function with "While True" format
# MODIFIED TO INSERT TWEET DATA DIRECTLY INTO MONGO

counter = 1

# Create list to store responses
# json_output =[]

# Loop 'while True:' runs until code is interrupted or API request limit exceeded

while True:
    
    # Submit twitter request
    response = requests.request("GET", url, headers=headers)
    
    # Check and print status / first text message of json response to confirm unique pulls
    print(response.status_code)
    print(response.json()['data'][0]['text'])
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
        print(response.status_code)
    
    # Save JSON response to Mongo database (TweetsAPI), collection (tweets_db)
    db.tweets_db.insert_many(response.json()['data'])
    
    # Check to see if 'next_token' in JSON response metatdata; if so, reset 'next' to iterate; if not, break
    if 'next_token' not in response.json()['meta']:
        break
    next = f"next_token={response.json()['meta']['next_token']}"
    print(next)
    print(f"Request: {counter}")
    print(f"Tweets pulled: {counter*100}")
    print('-------------------------------------------------')
    
    # Pause iteration for 20 seconds to allow unique next_token to generate and json to load
    time.sleep(20)
    
    # Iterate counter
    counter += 1

    
print("")
print("-----------------")
print("API Query Results")
print("-----------------")
print(f"Pages:            {counter}")
print(f"Tweets per Page:  {len(json_output[0]['data'])}")


200
Man, we were ready to play with COVID positive players.

Fixed that for ya, Trevor! https://t.co/DxgpLNroKr
next_token=b26v89c19zqg8o3foserwrhz7abnodot0h34o97ktsqrh
Request: 1
Tweets pulled: 100
-------------------------------------------------
200
@4_ever_Patriot @riverguy66 We are trying. First recall petition got stopped bc of Covid. Second one is circulating now
next_token=b26v89c19zqg8o3foserwrhz7abu5p6reguup39xexnct
Request: 2
Tweets pulled: 200
-------------------------------------------------
200
Might be of interest 👇

Supporting Young People’s Mental Health and Wellbeing - Covid-19 and beyond Tickets, Thu 3 Dec 2020 at 14:00 | Eventbrite https://t.co/Nl1orpE52S
next_token=b26v89c19zqg8o3foserwrhz7ajionqxg1c0jyqsza1dp
Request: 3
Tweets pulled: 300
-------------------------------------------------
200
“Quit smoking – A way to reduce the spread of COVID-19 in The Gambia” https://t.co/eWPh4wZy5x
next_token=b26v89c19zqg8o3foserwrhz7ar3y7a40i5jeg0qzrgcd
Request: 4
Tweets pulled

200
My aunt needs to stop throwing parties during COVID.

She works in a group home and she knows how delicate her clients are but she throws a party.
next_token=b26v89c19zqg8o3foserwri9nlcu8g3dsdzf1b3pozbzx
Request: 28
Tweets pulled: 2800
-------------------------------------------------
200
@CNNPolitics Fake news there is no covid
next_token=b26v89c19zqg8o3foserwri9nlkjsxi5ti6jgrvuitgn1
Request: 29
Tweets pulled: 2900
-------------------------------------------------
200
#Duke University #schools the country on how to stay open during the COVID-19 pandemic - Nov 21 @ 9:50 AM ET https://t.co/yIxw4cNrtm
next_token=b26v89c19zqg8o3foserwri9nlkrecqaorn5600r4n0jh
Request: 30
Tweets pulled: 3000
-------------------------------------------------
200
@oldlife @MZHemingway This is BS, these freaking liberals, they brought coronavirus over here just so they could cheat in this election it’s pretty clear now. They needed COVID for the absentee ballot‘s to work.....
next_token=b26v89c19zqg8o3fose

200
“Too many Americans believe in free-dumb now — the right, for example, to spread a deadly virus, as a heroic act of freedom.”

Do Americans Understand How Bad Covid in America Really Is? | by umair haque | Nov, 2020 | Eudaimonia and Co https://t.co/brIMyTfbFs
next_token=b26v89c19zqg8o3foserwri9s3qgezfa9us9xegzajo1p
Request: 55
Tweets pulled: 5500
-------------------------------------------------
200
@SamCarter3334 Cause Covid made it where the other characters couldn’t return. 😔
next_token=b26v89c19zqg8o3foserwri9s3y1q71nyfy5lvk54iyyl
Request: 56
Tweets pulled: 5600
-------------------------------------------------
200
@MichaelRDurham @BobKusterer @KMOV @DrSamPage Pretty sure the plan isn’t to eradicate Covid. Appears to be a Flu type approach. Vaccines &amp; antivirals. Will 30,000 deaths per year be ok or should we wear masks, lockdown &amp; mitigate until 0 deaths?...you know like we do with the flu
next_token=b26v89c19zqg8o3foserwri9s45p5yvq5x4gyzuldi7b1
Request: 57
Tweets pull

200
@TMZ Because there is no Covid in Tampa?
next_token=b26v89c19zqg8o3foserwri9v5en5ncwtv5jbc02aexkt
Request: 81
Tweets pulled: 8100
-------------------------------------------------
200
Treasury Secretary Mnuchin says he will not extend several federal COVID-19 relief programs https://t.co/IACweo5Ehs
next_token=b26v89c19zqg8o3foserwri9v5malpyjpur1h89oxygl9
Request: 82
Tweets pulled: 8200
-------------------------------------------------
200
Why can't they grow edible human meat from stem cells?

https://t.co/uHrvNG9oAS
next_token=b26v89c19zqg8o3foserwri9v5tvwh8mon47vry0pevzx
Request: 83
Tweets pulled: 8300
-------------------------------------------------
200
@realDonaldTrump how could you possible think that the American people would have any faith in you protecting us from Covid-19 when you can’t even keep yourself nor  your son @DonaldJTrumpJr safe?!?! #YouHadYourChance
next_token=b26v89c19zqg8o3foserwri9v61jcuss608qk9lizxtvh
Request: 84
Tweets pulled: 8400
-----------------------

200
@LEGO_Group @AwayDayJoe_ Get the young lad over when covid is away. Could be a great addition to the team ☺️
next_token=b26v89c19zqg8o3foserwri9y7i5u4n4jkl7iv89k527x
Request: 109
Tweets pulled: 10900
-------------------------------------------------
200
Heartbroken @CollegeGameDay is here in Norman and we aren’t down there breaking Gianna in. Pinche COVID.
next_token=b26v89c19zqg8o3foserwri9y7pudp3jc06cswby56gzh
Request: 110
Tweets pulled: 11000
-------------------------------------------------
200
We're currently on scene of a large unauthorised gathering in #Basildon town centre and have made arrests.

It's a clear &amp; blatant breach of the Government's COVID-19 regulations.

Attempts to engage with those attending have been unsuccessful so we're taking enforcement action. https://t.co/7HAymRbDpV
next_token=b26v89c19zqg8o3foserwri9y7xfo5izb8het5xa7xtvh
Request: 111
Tweets pulled: 11100
-------------------------------------------------
200
$COCP FDA Breakthrough Therapy Accelera

...

200
IT REALLY MAKES ME ANGRY WHEN I HEAR IGNORANT PEOPLE SAY “COVID IS JUST A FLU, IF YOU’RE HEALTY” I’m sorry to all the families suffering lose and heartbreak right now. It’s definitely NOT just a flu, and “the unhealthy” aren’t the only ones dying!
next_token=b26v89c19zqg8o3foserwvqix30atva5zitm2egl51j7h
Request: 1932
Tweets pulled: 193200
-------------------------------------------------
200
@realDonaldTrump 12 MILLION covid 19 cases on your watch YOU PATHETIC LOSER!  YOU ARE NOT A MAN  YOU ARE AN ORANGE MONKEY BOY!  GOD'S WILL BE DONE ON EARTH AS IT IS IN HEAVEN!!!!!!  BYE !!!!!!!!!!!!!!!!!!!!!!
next_token=b26v89c19zqg8o3foserwvqix37zdqkg2r7o2fpdpt9bx
Request: 1933
Tweets pulled: 193300
-------------------------------------------------
200
@YourboyQ254 Really sorry I’m not going to be able to meet you in vegas! This covid thing is hitting our family hard. Have a safe trip and enjoy your time with family!
next_token=b26v89c19zqg8o3foserwvqix385ug5dtivm36cdj2dml
Request: 1934
Tweets

200
Yes, threeis a fascist Bolsonaro dictator in Brazil. fascist Trump!s appirentice, the left view must bury this fascist in the ballot box. a fascist connot lead the Brazilian poeple, the bolsonaron writes covid number 19 on hers shirt. real virüs https://t.co/ghqlcBT2Ib
next_token=b26v89c19zqg8o3foserwvqj04gwaj4na8l7ha59hb4vx
Request: 1957
Tweets pulled: 195700
-------------------------------------------------
200
Bedlam is today which means I will not be leaving my house bc covid is everywhere in norman 😭
next_token=b26v89c19zqg8o3foserwvqj04olv0gc1d35bhihi7x4t
Request: 1958
Tweets pulled: 195800
-------------------------------------------------
200
@conspiraboomers yes means i get a covid vaccine right? then yes.
next_token=b26v89c19zqg8o3foserwvqj04w89kbz1e06alwwio3gd
Request: 1959
Tweets pulled: 195900
-------------------------------------------------
200
So i gots the fever and it went away al7amdllah. Now i gotta get tested for covid, and it’s my first one so im kinda nervous,

200
@mydoxies7 @wdunlap @GOPLeader @Mermaid_singing You are being dishonest. First off there isn't 250k deaths from covid there is 250k death with covid.
next_token=b26v89c19zqg8o3foserwvqj36568xz9z2g9q6bl75ugt
Request: 1983
Tweets pulled: 198300
-------------------------------------------------
200
@realDonaldTrump AMERICA needs a leader to deal with 2000 daily American deaths due to your DISASTEROUS response to COVID. Go back to your golfing...

https://t.co/xgKYz7so20
next_token=b26v89c19zqg8o3foserwvqj36ctopkwmk982gzoysgot
Request: 1984
Tweets pulled: 198400
-------------------------------------------------
200
Stanford faculty condemn Dr Scott Atlas for spreading misinformation about the pandemic https://t.co/Uf1r9BJmli
next_token=b26v89c19zqg8o3foserwvqj4n1zh1cogok17zloeuvlp
Request: 1985
Tweets pulled: 198500
-------------------------------------------------
200
I made a trip for 3 months stayed  with family went to Wal-Mart for groceries mask on hugging my grand children who ne

200
@AvrilLavigne representing how we feel about Covid! https://t.co/rzNAuluxf8
next_token=b26v89c19zqg8o3foserwvqtkxg3cf9gxz6tuooo1j8u5
Request: 2008
Tweets pulled: 200800
-------------------------------------------------
200
Harvest Gala COVID style. Thank you to the @PerthCountyAg &amp; sponsors for making a gala in a box happen. #HarvestGala2020 #ontag #loveONTfood #ruralon #perthcty https://t.co/bAJCzUJl7C
next_token=b26v89c19zqg8o3foserwvqtkxgd2k0s207c66uyo5k71
Request: 2009
Tweets pulled: 200900
-------------------------------------------------
200
This girl I follow on Instagram has complained about wearing a mask for months. Tonight on her story she said, “My BFF has Covid and is in the hospital. Yes this virus is real. But closing restaurants and wearing masks is bullshit” 
What. Is. Wrong. With. Her.
next_token=b26v89c19zqg8o3foserwvqtkxo0hvbnshx3x3x9oddod
Request: 2010
Tweets pulled: 201000
-------------------------------------------------
200
I’m old enough to remember whe

KeyboardInterrupt: 

In [30]:
# Check tweets_db collection; confirm on tweet per id (query on tweet id; should only return one tweet)

db.tweets_db.find_one()


{'_id': ObjectId('5fb91de569accb7704c9694d'),
 'author_id': '28587570',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0},
 'id': '1330149477081034756',
 'text': "Scientists don't know why some young, healthy adults experience 'long COVID'. https://t.co/geEdt30Q8J —\xa0via @healthing_ca",
 'lang': 'en',
 'created_at': '2020-11-21T14:02:02.000Z'}

In [41]:
# Check to confirm only one tweet is stored per test query (find one specific tweet id)

for object in db.tweets_db.find({'id': '1330149477081034756'}):  pprint.pprint(object)

{'_id': ObjectId('5fb91de569accb7704c9694d'),
 'author_id': '28587570',
 'created_at': '2020-11-21T14:02:02.000Z',
 'id': '1330149477081034756',
 'lang': 'en',
 'public_metrics': {'like_count': 0,
                    'quote_count': 0,
                    'reply_count': 0,
                    'retweet_count': 0},
 'text': "Scientists don't know why some young, healthy adults experience "
         "'long COVID'. https://t.co/geEdt30Q8J —\xa0via @healthing_ca"}


In [60]:
# Review organized data
df = pd.DataFrame(tweet_data)
df.tail()

,ID,Date,Retweets,Replies,Likes,Quotes,Tweet
10047,1329852691141443584,2020-11-20T18:22:43.000Z,0,0,0,0,"According to the latest COVID-19 data, 250,000..."
10048,1329852691032510466,2020-11-20T18:22:43.000Z,0,0,0,0,I work answering an RN COVID hotline and can v...
10049,1329852690923479042,2020-11-20T18:22:43.000Z,0,0,0,0,The Broncos announced that Sunday will be thei...
10050,1329852689539362817,2020-11-20T18:22:43.000Z,0,0,0,0,"@Stats_Burger @numbers_truth ""die from COVID-1..."
10051,1329852689493049344,2020-11-20T18:22:43.000Z,0,0,0,0,Lmao well why would you? Stupid Americans have...


In [61]:
# Filter by mask
mask_df = df[df['Tweet'].str.contains('mask|face covering')]
mask_df

,ID,Date,Retweets,Replies,Likes,Quotes,Tweet
36,1329638955268993025,2020-11-20T04:13:25.000Z,0,0,0,0,@DarciSings @Zigmanfreud @GavinNewsom He’s a c...
55,1329639128007249921,2020-11-20T04:14:06.000Z,0,0,0,0,Top WHO official in Europe says face masks cou...
67,1329639209666301952,2020-11-20T04:14:25.000Z,0,0,0,0,"For the love of God, everyone: Wear your masks..."
80,1329639385864704000,2020-11-20T04:15:07.000Z,0,0,0,0,@JoeBiden If masks stop the spread of COVID it...
84,1329639383260082176,2020-11-20T04:15:07.000Z,0,0,0,0,"#FridayThoughts BEEN MASKING FOR MONTHS, Yet s..."
...,...,...,...,...,...,...,...
9982,1329852187934154753,2020-11-20T18:20:43.000Z,0,0,0,0,How many Republican politicians vs Democrat po...
10002,1329852356369010691,2020-11-20T18:21:23.000Z,0,0,0,0,Covid gadget alert: Think your trusty mask nee...
10005,1329852354334765061,2020-11-20T18:21:23.000Z,0,0,0,0,@UHOHitsMark @drmkry @aVoice4MA6 @Don_Fabbri T...
10027,1329852525831327746,2020-11-20T18:22:04.000Z,0,0,0,0,@GatoPolitico5 @tvanhorbeck @cnnbrk I AM SO BE...


In [62]:
# Filter by vaccine
vaccine_df = df[df['Tweet'].str.contains('vaccine')]
vaccine_df

,ID,Date,Retweets,Replies,Likes,Quotes,Tweet
14,1329638788507856897,2020-11-20T04:12:45.000Z,0,0,0,0,Coronavirus | Oxford University COVID-19 vacci...
22,1329638876877516800,2020-11-20T04:13:06.000Z,0,0,0,0,Serum Institute of India’s CEO Adar Poonawalla...
41,1329639050387496960,2020-11-20T04:13:47.000Z,0,0,0,0,@berrybram @channel5_tv Some things come back ...
49,1329639039348170752,2020-11-20T04:13:45.000Z,0,0,0,0,@NicolaSturgeon\n@ScotGovFM\n@theSNP\nWATCH fi...
69,1329639205337788416,2020-11-20T04:14:24.000Z,0,0,0,0,Covid vaccine to send flight costs skyrocketin...
...,...,...,...,...,...,...,...
9871,1329851166335721473,2020-11-20T18:16:40.000Z,0,0,0,0,@IndianExpress Check out this blog to know wha...
9902,1329851508821733378,2020-11-20T18:18:01.000Z,0,0,0,0,Geraldo Rivera suggests naming COVID-19 vaccin...
9962,1329852017112543232,2020-11-20T18:20:02.000Z,0,0,0,0,"Covid vaccine How Indian airlines, airports ar..."
9989,1329852182623969281,2020-11-20T18:20:42.000Z,0,0,0,0,@PSformaldehyde @EdwardNorton @neal_katyal @Pr...


In [63]:
# Filter by lockdown
lockdown_df = df[df['Tweet'].str.contains('lockdown|quarantine|stay at home')]
lockdown_df

,ID,Date,Retweets,Replies,Likes,Quotes,Tweet
16,1329638784229646336,2020-11-20T04:12:44.000Z,0,0,0,0,@crankyoldman13 @ladalavara @dallasrbaird This...
30,1329638962546307073,2020-11-20T04:13:26.000Z,0,0,0,0,@CP24 If we have the most strict lockdown at t...
55,1329639128007249921,2020-11-20T04:14:06.000Z,0,0,0,0,Top WHO official in Europe says face masks cou...
84,1329639383260082176,2020-11-20T04:15:07.000Z,0,0,0,0,"#FridayThoughts BEEN MASKING FOR MONTHS, Yet s..."
88,1329639380726673409,2020-11-20T04:15:06.000Z,0,0,0,0,@DebiePritchard @GavinNewsom It must physicall...
...,...,...,...,...,...,...,...
9868,1329851166813941760,2020-11-20T18:16:40.000Z,0,0,0,0,France 'still far' from lifting Covid-19 lockd...
9958,1329851927157469188,2020-11-20T18:19:41.000Z,0,0,0,0,Where can I go on holiday after lockdown? #ukl...
9963,1329852016852611074,2020-11-20T18:20:02.000Z,0,0,0,0,Newark officials say residents need to “stay a...
9976,1329852101577625600,2020-11-20T18:20:23.000Z,0,0,0,0,@sc_wadsy Boris J seems to be determined to de...


In [64]:
# Filter by social distancing
distance_df = df[df['Tweet'].str.contains('social distance|social distancing')]
distance_df

,ID,Date,Retweets,Replies,Likes,Quotes,Tweet
88,1329639380726673409,2020-11-20T04:15:06.000Z,0,0,0,0,@DebiePritchard @GavinNewsom It must physicall...
188,1329640226898132997,2020-11-20T04:18:28.000Z,0,0,0,0,@1997Aggies @SylvesterL_ @kayleighmcenany @Gov...
329,1329641413592698886,2020-11-20T04:23:11.000Z,0,0,1,0,Did someone say...a panda-emic dining experien...
353,1329641673656242189,2020-11-20T04:24:13.000Z,0,0,0,0,"Wear a mask, social distance, and be mindful. ..."
980,1329647016289337345,2020-11-20T04:45:26.000Z,0,0,0,0,I hope you selfish motherfuckers who don’t tak...
1193,1329648796033196035,2020-11-20T04:52:31.000Z,0,0,0,0,Second wave of covid has hit delhi very badly ...
1547,1329754148489875457,2020-11-20T11:51:09.000Z,0,0,0,0,@redwoodricker @katnandu @MaddowBlog @KamalaHa...
1737,1329782113198071808,2020-11-20T13:42:16.000Z,0,0,0,0,@Lilacarn_ @rob_miller12345 According to COVID...
1950,1329783982855172099,2020-11-20T13:49:42.000Z,0,0,0,0,@MattGertz Why not call it the anti-Trump vacc...
2032,1329784662512848901,2020-11-20T13:52:24.000Z,0,0,0,0,pls stay safe wear mask and keep social distan...


In [65]:
# Push dataframes to CSV

df.to_csv('../Twitter_Data/tweet_data1.csv', index=False)
mask_df.to_csv('../Twitter_Data/mask_data1.csv', index=False)
vaccine_df.to_csv('../Twitter_Data/vaccine_data1.csv', index=False)
lockdown_df.to_csv('../Twitter_Data/lockdown_data1.csv', index=False)
distance_df.to_csv('../Twitter_Data/distance_data1.csv', index=False)